In [59]:
!pip install geocoder

In [60]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
from sklearn.metrics import silhouette_score

%matplotlib notebook

print('All libraries imported.')

All libraries imported.


In [61]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai')[-1]
df.rename(columns={'Area': 'Neighborhood'}, inplace=True)
df.head(10)

,Neighborhood,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,Chakala,"Andheri,Western Suburbs",19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270
5,Marol,"Andheri,Western Suburbs",19.119219,72.882743
6,Sahar,"Andheri,Western Suburbs",19.098889,72.867222
7,Seven Bungalows,"Andheri,Western Suburbs",19.129052,72.817018
8,Versova,"Andheri,Western Suburbs",19.120000,72.820000
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.284167,72.871111


In [62]:
df['Location'].value_counts()

South Mumbai                       30
Andheri,Western Suburbs             9
Eastern Suburbs                     4
Western Suburbs                     4
Bandra,Western Suburbs              3
Kandivali West,Western Suburbs      3
Powai,Eastern Suburbs               3
Mira-Bhayandar,Western Suburbs      3
Ghatkopar,Eastern Suburbs           3
Kalbadevi,South Mumbai              2
Harbour Suburbs                     2
Mumbai                              2
Malad,Western Suburbs               2
Khar,Western Suburbs                2
Borivali (West),Western Suburbs     2
Goregaon,Western Suburbs            2
Vile Parle,Western Suburbs          1
Colaba,South Mumbai                 1
Tardeo,South Mumbai                 1
Kamathipura,South Mumbai            1
Fort,South Mumbai                   1
Dadar,South Mumbai                  1
Antop Hill,South Mumbai             1
Byculla,South Mumbai                1
Sanctacruz,Western Suburbs          1
Trombay,Harbour Suburbs             1
Govandi,Harb

In [63]:
df['Location'] = df['Location'].apply(lambda x: x.split(',')[-1])
df.head(10)

,Neighborhood,Location,Latitude,Longitude
0,Amboli,Western Suburbs,19.129300,72.843400
1,Chakala,Western Suburbs,19.111388,72.860833
2,D.N. Nagar,Western Suburbs,19.124085,72.831373
3,Four Bungalows,Western Suburbs,19.124714,72.827210
4,Lokhandwala,Western Suburbs,19.130815,72.829270
5,Marol,Western Suburbs,19.119219,72.882743
6,Sahar,Western Suburbs,19.098889,72.867222
7,Seven Bungalows,Western Suburbs,19.129052,72.817018
8,Versova,Western Suburbs,19.120000,72.820000
9,Mira Road,Western Suburbs,19.284167,72.871111


In [64]:
df['Location'].value_counts()

South Mumbai       39
Western Suburbs    33
Eastern Suburbs    12
Harbour Suburbs     4
Mumbai              2
Name: Location, dtype: int64

In [65]:
df['Latitude1'] = None
df['Longitude1'] = None

for i, neigh in enumerate(df['Neighborhood']):
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neigh))
        lat_lng_coords = g.latlng

    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]

    df.loc[i, 'Latitude1'] = latitude
    df.loc[i, 'Longitude1'] = longitude

df.head(10)

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1
0,Amboli,Western Suburbs,19.129300,72.843400,19.12906,72.84644
1,Chakala,Western Suburbs,19.111388,72.860833,19.10836,72.86236
2,D.N. Nagar,Western Suburbs,19.124085,72.831373,19.12505,72.83248
3,Four Bungalows,Western Suburbs,19.124714,72.827210,19.1264,72.8242
4,Lokhandwala,Western Suburbs,19.130815,72.829270,19.14321,72.82497
5,Marol,Western Suburbs,19.119219,72.882743,19.11905,72.88281
6,Sahar,Western Suburbs,19.098889,72.867222,19.109461,72.852807
7,Seven Bungalows,Western Suburbs,19.129052,72.817018,19.12857,72.8214
8,Versova,Western Suburbs,19.120000,72.820000,19.13769,72.81348
9,Mira Road,Western Suburbs,19.284167,72.871111,19.262969,72.87265


In [66]:
df['Latdiff'] = abs(df['Latitude'] - df['Latitude1'])
df['Longdiff'] = abs(df['Longitude'] - df['Longitude1'])
df.head(10)

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,Amboli,Western Suburbs,19.129300,72.843400,19.12906,72.84644,0.00024,0.00304
1,Chakala,Western Suburbs,19.111388,72.860833,19.10836,72.86236,0.003028,0.001527
2,D.N. Nagar,Western Suburbs,19.124085,72.831373,19.12505,72.83248,0.000965,0.001107
3,Four Bungalows,Western Suburbs,19.124714,72.827210,19.1264,72.8242,0.001686,0.00301
4,Lokhandwala,Western Suburbs,19.130815,72.829270,19.14321,72.82497,0.012395,0.0043
5,Marol,Western Suburbs,19.119219,72.882743,19.11905,72.88281,0.000169,0.000067
6,Sahar,Western Suburbs,19.098889,72.867222,19.109461,72.852807,0.010572,0.014415
7,Seven Bungalows,Western Suburbs,19.129052,72.817018,19.12857,72.8214,0.000482,0.004382
8,Versova,Western Suburbs,19.120000,72.820000,19.13769,72.81348,0.01769,0.00652
9,Mira Road,Western Suburbs,19.284167,72.871111,19.262969,72.87265,0.021198,0.001539


In [67]:
df.loc[df.Latdiff>0.001, 'Latitude'] = df.loc[df.Latdiff>0.001, 'Latitude1']
df.loc[df.Longdiff>0.001, 'Longitude'] = df.loc[df.Longdiff>0.001, 'Longitude1']
df.head(10)

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,Amboli,Western Suburbs,19.1293,72.84644,19.12906,72.84644,0.00024,0.00304
1,Chakala,Western Suburbs,19.10836,72.86236,19.10836,72.86236,0.003028,0.001527
2,D.N. Nagar,Western Suburbs,19.124085,72.83248,19.12505,72.83248,0.000965,0.001107
3,Four Bungalows,Western Suburbs,19.1264,72.8242,19.1264,72.8242,0.001686,0.00301
4,Lokhandwala,Western Suburbs,19.14321,72.82497,19.14321,72.82497,0.012395,0.0043
5,Marol,Western Suburbs,19.119219,72.882743,19.11905,72.88281,0.000169,0.000067
6,Sahar,Western Suburbs,19.109461,72.852807,19.109461,72.852807,0.010572,0.014415
7,Seven Bungalows,Western Suburbs,19.129052,72.8214,19.12857,72.8214,0.000482,0.004382
8,Versova,Western Suburbs,19.13769,72.81348,19.13769,72.81348,0.01769,0.00652
9,Mira Road,Western Suburbs,19.262969,72.87265,19.262969,72.87265,0.021198,0.001539


In [68]:
df.where(df['Latitude']==df['Latitude1'])

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chakala,Western Suburbs,19.10836,72.86236,19.10836,72.86236,0.003028,0.001527
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Four Bungalows,Western Suburbs,19.1264,72.8242,19.1264,72.8242,0.001686,0.00301
4,Lokhandwala,Western Suburbs,19.14321,72.82497,19.14321,72.82497,0.012395,0.0043
...,...,...,...,...,...,...,...,...
85,Parel,South Mumbai,18.9959,72.84,18.9959,72.83923,0.0059,0.00077
86,Gowalia Tank,South Mumbai,18.96449,72.81114,18.96449,72.81114,0.00204,0.001437
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,Dharavi,Mumbai,19.04674,72.85462,19.04674,72.85462,0.006532,0.00377


In [69]:
df.where(df['Longitude']==df['Longitude1'])

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,Amboli,Western Suburbs,19.1293,72.84644,19.12906,72.84644,0.00024,0.00304
1,Chakala,Western Suburbs,19.10836,72.86236,19.10836,72.86236,0.003028,0.001527
2,D.N. Nagar,Western Suburbs,19.124085,72.83248,19.12505,72.83248,0.000965,0.001107
3,Four Bungalows,Western Suburbs,19.1264,72.8242,19.1264,72.8242,0.001686,0.00301
4,Lokhandwala,Western Suburbs,19.14321,72.82497,19.14321,72.82497,0.012395,0.0043
...,...,...,...,...,...,...,...,...
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,Gowalia Tank,South Mumbai,18.96449,72.81114,18.96449,72.81114,0.00204,0.001437
87,Dava Bazaar,South Mumbai,18.946882,72.82958,18.94684,72.82958,0.000042,0.001782
88,Dharavi,Mumbai,19.04674,72.85462,19.04674,72.85462,0.006532,0.00377


In [70]:
df.drop(['Latitude1', 'Longitude1', 'Latdiff', 'Longdiff'], axis=1, inplace=True)
df.head(10)

,Neighborhood,Location,Latitude,Longitude
0,Amboli,Western Suburbs,19.1293,72.84644
1,Chakala,Western Suburbs,19.10836,72.86236
2,D.N. Nagar,Western Suburbs,19.124085,72.83248
3,Four Bungalows,Western Suburbs,19.1264,72.8242
4,Lokhandwala,Western Suburbs,19.14321,72.82497
5,Marol,Western Suburbs,19.119219,72.882743
6,Sahar,Western Suburbs,19.109461,72.852807
7,Seven Bungalows,Western Suburbs,19.129052,72.8214
8,Versova,Western Suburbs,19.13769,72.81348
9,Mira Road,Western Suburbs,19.262969,72.87265


In [71]:
neighborhoods_mumbai = df.groupby('Location')['Neighborhood'].nunique()
neighborhoods_mumbai

Location
Eastern Suburbs    12
Harbour Suburbs     4
Mumbai              2
South Mumbai       39
Western Suburbs    33
Name: Neighborhood, dtype: int64

In [72]:
address = 'Mumbai, IN'
geolocator = Nominatim(user_agent="Untitled")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Mumbai are 19.0785451, 72.878176.


In [73]:
map_mumbai= folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, location, neighborhood in zip(df['Latitude'], df['Longitude'], df['Location'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)

map_mumbai

In [74]:
neighborhood_name = df.loc[0, 'Neighborhood']
neighborhood_lat = df.loc[0, 'Latitude']
neighborhood_long = df.loc[0, 'Longitude']
print("The neighborhood is {} and it's geographical coordinates are {} latitude and {} longitude".format(neighborhood_name,neighborhood_lat, neighborhood_long))

The neighborhood is Amboli and it's geographical coordinates are 19.1293 latitude and 72.84644000000003 longitude


In [75]:
CLIENT_ID = '2GQBW5PR0QFXTOGCHKTRFWJBTGOFOHXW1TRTNRAFURQ5FE1X'
CLIENT_SECRET = '3QH40WMZIIDSQN1RFAVAEQHUIMOQUJPKYPABQVNTSDQJN2YD'
VERSION = 20202808
radius = 1000
LIMIT = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_lat,
    neighborhood_long,
    radius,
    LIMIT)

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '64acf4e14a990b72df35e2eb'},
 'response': {'queryRefinements': {'target': {'type': 'path',
    'url': '/venue/explore',
    'params': {'ll': '19.129300,72.846440', 'radius': '1000'}},
   'refinements': [{'query': 'Food'},
    {'query': 'Nightlife'},
    {'query': 'Coffee'},
    {'query': 'Shops'},
    {'query': 'Arts'},
    {'query': 'Outdoors'}]},
  'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Jogeshwari West',
  'headerFullLocation': 'Jogeshwari West, Mumbai',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 19.13830000900001,
    'lng': 72.85594823590122},
   'sw': {'lat': 19.120299990999992, 'lng': 72.83693176409884}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
      

In [77]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-78-dc733077f29a>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  nearby_venues = json_normalize(venues)


,name,categories,lat,lng
0,Cafe Arfa,Indian Restaurant,19.128930,72.847140
1,"5 Spice , Bandra",Chinese Restaurant,19.130421,72.847206
2,Jaffer Bhai's Delhi Darbar,Mughlai Restaurant,19.137714,72.845909
3,Domino's Pizza,Pizzeria,19.131000,72.848000
4,Shawarma Factory,Falafel Restaurant,19.124591,72.840398


In [79]:
print("{} venues were returned for {} by Foursquare".format(len(nearby_venues), neighborhood_name))

31 venues were returned for Amboli by Foursquare


In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return(nearby_venues)

In [81]:
mumbai_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=radius)
print(mumbai_venues.shape)
mumbai_venues.head(10)

(3589, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.1293,72.84644,Cafe Arfa,19.128930,72.847140,Indian Restaurant
1,Amboli,19.1293,72.84644,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
2,Amboli,19.1293,72.84644,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant
3,Amboli,19.1293,72.84644,Domino's Pizza,19.131000,72.848000,Pizzeria
4,Amboli,19.1293,72.84644,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
5,Amboli,19.1293,72.84644,Garden Court,19.127188,72.837478,Indian Restaurant
6,Amboli,19.1293,72.84644,Narayan Sandwich,19.121398,72.850270,Sandwich Spot
7,Amboli,19.1293,72.84644,Persia Darbar,19.136952,72.846822,Indian Restaurant
8,Amboli,19.1293,72.84644,Subway,19.127860,72.844461,Sandwich Spot
9,Amboli,19.1293,72.84644,Courtyard Restaurant & Bar,19.131692,72.837269,Asian Restaurant


In [82]:
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")
mumbai_onehot.head()

,Airport,Airport Terminal,American Restaurant,Antique Store,Apartment or Condo,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Arts and Entertainment,...,Travel and Transportation,Vegan and Vegetarian Restaurant,Video Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Store,Women's Store,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood']
mumbai_onehot.head()

,Airport,Airport Terminal,American Restaurant,Antique Store,Apartment or Condo,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Arts and Entertainment,...,Travel and Transportation,Vegan and Vegetarian Restaurant,Video Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Store,Women's Store,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood', sort=False).mean().reset_index()
print(mumbai_grouped.shape)
mumbai_grouped.head(10)

(90, 211)


,Neighborhood,Airport,Airport Terminal,American Restaurant,Antique Store,Apartment or Condo,Aquarium,Arcade,Arepa Restaurant,Art Gallery,...,Travel and Transportation,Vegan and Vegetarian Restaurant,Video Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Store,Women's Store,Zoo
0,Amboli,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,...,0.064516,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Chakala,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.025,0.0,...,0.000000,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,D.N. Nagar,0.0,0.0,0.000000,0.0,0.023810,0.0,0.0,0.000,0.0,...,0.000000,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0
3,Four Bungalows,0.0,0.0,0.000000,0.0,0.016949,0.0,0.0,0.000,0.0,...,0.000000,0.033898,0.0,0.0,0.0,0.0,0.0,0.0,0.016949,0.0
4,Lokhandwala,0.0,0.0,0.012048,0.0,0.000000,0.0,0.0,0.000,0.0,...,0.000000,0.012048,0.0,0.0,0.0,0.0,0.0,0.0,0.012048,0.0
5,Marol,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
6,Sahar,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,...,0.000000,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
7,Seven Bungalows,0.0,0.0,0.000000,0.0,0.016393,0.0,0.0,0.000,0.0,...,0.000000,0.032787,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.0
8,Versova,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9,Mira Road,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,...,0.066667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0


In [85]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amboli----
               venue  freq
0  Indian Restaurant  0.13
1        Coffee Shop  0.06
2             Bakery  0.06
3           Pizzeria  0.06
4   Asian Restaurant  0.06


----Chakala----
               venue  freq
0  Indian Restaurant  0.18
1              Hotel  0.18
2               Café  0.08
3           Pizzeria  0.05
4      Movie Theater  0.05


----D.N. Nagar----
                venue  freq
0                 Bar  0.17
1   Indian Restaurant  0.10
2                 Pub  0.10
3            Pizzeria  0.07
4  Chinese Restaurant  0.05


----Four Bungalows----
                venue  freq
0                 Pub  0.07
1                Café  0.07
2   Indian Restaurant  0.07
3  Chinese Restaurant  0.07
4                 Bar  0.05


----Lokhandwala----
                venue  freq
0   Indian Restaurant  0.13
1                Café  0.07
2  Chinese Restaurant  0.06
3                 Pub  0.05
4  Italian Restaurant  0.04


----Marol----
               venue  freq
0  Indian Restaurant  0.42
1

In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Indian Restaurant,Coffee Shop,Bakery,Pizzeria,Asian Restaurant,Travel and Transportation,Sandwich Spot,Bar,Metro Station,Tea Room
1,Chakala,Indian Restaurant,Hotel,Café,Pizzeria,Movie Theater,Hotel Bar,Fast Food Restaurant,Restaurant,Asian Restaurant,Vegan and Vegetarian Restaurant
2,D.N. Nagar,Bar,Indian Restaurant,Pub,Pizzeria,Chinese Restaurant,Juice Bar,Snack Place,Vegan and Vegetarian Restaurant,Coffee Shop,Bowling Alley
3,Four Bungalows,Pub,Café,Indian Restaurant,Chinese Restaurant,Bar,Ice Cream Parlor,Seafood Restaurant,Pizzeria,Coffee Shop,Vegan and Vegetarian Restaurant
4,Lokhandwala,Indian Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Coffee Shop,Bakery,Bar,Fast Food Restaurant,Asian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
85,Parel,Indian Restaurant,Office,Chinese Restaurant,Coffee Shop,Plaza,Maharashtrian Restaurant,Roof Deck,Bar,Bakery,Restaurant
86,Gowalia Tank,Indian Restaurant,Bakery,Café,Fast Food Restaurant,Sandwich Spot,Coffee Shop,Chinese Restaurant,Pizzeria,Snack Place,Theater
87,Dava Bazaar,Indian Restaurant,Dessert Shop,Chinese Restaurant,Café,Restaurant,Fast Food Restaurant,Arts and Entertainment,Rail Station,Bar,Bakery
88,Dharavi,Indian Restaurant,Café,Vegan and Vegetarian Restaurant,Bakery,Shopping Mall,Shoe Store,Seafood Restaurant,Garden,Movie Theater,Sandwich Spot


In [88]:
def plot(x, y):
    fig = plt.figure(figsize=(12,6))
    plt.plot(x, y, 'o-')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Scores')
    plt.title('Checking Optimum Number of Clusters')
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [89]:
maxk = 15
scores = []
kval = []

for k in range(2, maxk+1):
    cl_df = mumbai_grouped.drop('Neighborhood', axis=1)
    kmeans = KMeans(n_clusters=k).fit_predict(cl_df) #Choose any random_state

    score = silhouette_score(cl_df, kmeans, metric='euclidean', random_state=0)
    kval.append(k)
    scores.append(score)

print(scores)
print(kval)
plt.plot(kval, scores)
plt.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

[0.07396305606621259, 0.08622655797314803, 0.09012599944329783, 0.11406837446552628, 0.09999993139459837, 0.06839955635116196, 0.07265297672449196, 0.12425355486396161, 0.08671558353843774, 0.10734753475112081, 0.07184047059376172, 0.051708844799700435, 0.07784980858724763, 0.07829327662513763]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [90]:
k = 5
mum_clustering = mumbai_grouped.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=k, init="k-means++", random_state=40).fit(mum_clustering) #Can choose any random_state
kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3,
       1, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 0, 3, 1, 3, 0, 1, 1, 3, 3, 3, 3,
       1, 3, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 0, 1, 1, 1, 3, 3, 1, 1, 1, 3,
       3, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 4, 3, 3, 3, 3, 3, 3, 3, 1, 3,
       1, 0], dtype=int32)

In [91]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
mum_merged = df
mum_merged = mum_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
print(mum_merged.shape)
mum_merged

(90, 15)


,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Western Suburbs,19.1293,72.84644,1,Indian Restaurant,Coffee Shop,Bakery,Pizzeria,Asian Restaurant,Travel and Transportation,Sandwich Spot,Bar,Metro Station,Tea Room
1,Chakala,Western Suburbs,19.10836,72.86236,3,Indian Restaurant,Hotel,Café,Pizzeria,Movie Theater,Hotel Bar,Fast Food Restaurant,Restaurant,Asian Restaurant,Vegan and Vegetarian Restaurant
2,D.N. Nagar,Western Suburbs,19.124085,72.83248,1,Bar,Indian Restaurant,Pub,Pizzeria,Chinese Restaurant,Juice Bar,Snack Place,Vegan and Vegetarian Restaurant,Coffee Shop,Bowling Alley
3,Four Bungalows,Western Suburbs,19.1264,72.8242,1,Pub,Café,Indian Restaurant,Chinese Restaurant,Bar,Ice Cream Parlor,Seafood Restaurant,Pizzeria,Coffee Shop,Vegan and Vegetarian Restaurant
4,Lokhandwala,Western Suburbs,19.14321,72.82497,1,Indian Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Coffee Shop,Bakery,Bar,Fast Food Restaurant,Asian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Parel,South Mumbai,18.9959,72.84,3,Indian Restaurant,Office,Chinese Restaurant,Coffee Shop,Plaza,Maharashtrian Restaurant,Roof Deck,Bar,Bakery,Restaurant
86,Gowalia Tank,South Mumbai,18.96449,72.81114,1,Indian Restaurant,Bakery,Café,Fast Food Restaurant,Sandwich Spot,Coffee Shop,Chinese Restaurant,Pizzeria,Snack Place,Theater
87,Dava Bazaar,South Mumbai,18.946882,72.82958,3,Indian Restaurant,Dessert Shop,Chinese Restaurant,Café,Restaurant,Fast Food Restaurant,Arts and Entertainment,Rail Station,Bar,Bakery
88,Dharavi,Mumbai,19.04674,72.85462,1,Indian Restaurant,Café,Vegan and Vegetarian Restaurant,Bakery,Shopping Mall,Shoe Store,Seafood Restaurant,Garden,Movie Theater,Sandwich Spot


In [92]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [93]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Bhandup,Eastern Suburbs,Rail Station,Ice Cream Parlor,Asian Restaurant,Fast Food Restaurant,Airport,Music Venue,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument,Mosque
37,Kanjurmarg,Eastern Suburbs,Rail Station,Cupcake Shop,Food Truck,Fuel Station,Asian Restaurant,Movie Theater,Gift Store,Chinese Restaurant,Airport,Music Store
56,Cotton Green,South Mumbai,Pizzeria,Plaza,Movie Theater,Fast Food Restaurant,Rail Station,Vegan and Vegetarian Restaurant,Bakery,Ice Cream Parlor,Other Great Outdoors,Office Supply Store
89,Thane,Mumbai,Pizzeria,Performing Arts Venue,Fuel Station,Arts and Entertainment,Music Store,Mobile Phone Store,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument,Mosque


In [94]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Western Suburbs,Indian Restaurant,Coffee Shop,Bakery,Pizzeria,Asian Restaurant,Travel and Transportation,Sandwich Spot,Bar,Metro Station,Tea Room
2,D.N. Nagar,Western Suburbs,Bar,Indian Restaurant,Pub,Pizzeria,Chinese Restaurant,Juice Bar,Snack Place,Vegan and Vegetarian Restaurant,Coffee Shop,Bowling Alley
3,Four Bungalows,Western Suburbs,Pub,Café,Indian Restaurant,Chinese Restaurant,Bar,Ice Cream Parlor,Seafood Restaurant,Pizzeria,Coffee Shop,Vegan and Vegetarian Restaurant
4,Lokhandwala,Western Suburbs,Indian Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Coffee Shop,Bakery,Bar,Fast Food Restaurant,Asian Restaurant
6,Sahar,Western Suburbs,Fast Food Restaurant,Indian Restaurant,Hotel,Snack Place,Tea Room,Café,Vegan and Vegetarian Restaurant,Movie Theater,Chinese Restaurant,Shopping Mall
7,Seven Bungalows,Western Suburbs,Café,Bar,Pub,Indian Restaurant,Ice Cream Parlor,Chinese Restaurant,Seafood Restaurant,Pizzeria,Vegan and Vegetarian Restaurant,Coffee Shop
8,Versova,Western Suburbs,Café,Ice Cream Parlor,Beach,Bar,Coffee Shop,Chinese Restaurant,Pub,Smoke Shop,Frozen Yogurt Shop,Music Venue
9,Mira Road,Western Suburbs,Movie Theater,Convenience Store,Bar,Gift Store,Food Truck,Food Court,Multicuisine Indian Restaurant,Fast Food Restaurant,Electronics Store,College and University
10,Bhayandar,Western Suburbs,Bakery,Restaurant,Travel and Transportation,Bike Rental,Electronics Store,Diner,Rail Station,Food Truck,Indian Restaurant,Ice Cream Parlor
12,Bandstand Promenade,Western Suburbs,Coffee Shop,Café,Chinese Restaurant,Indian Restaurant,Snack Place,Tea Room,Pizzeria,Scenic Lookout,Performing Arts Venue,Deli


In [95]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Uttan,Western Suburbs,Resort,Water Park,Beach,Airport,Music Venue,Mobile Phone Store,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument,Mosque


In [96]:
mum_merged.loc[mum_merged['Cluster Labels'] == 3, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chakala,Western Suburbs,Indian Restaurant,Hotel,Café,Pizzeria,Movie Theater,Hotel Bar,Fast Food Restaurant,Restaurant,Asian Restaurant,Vegan and Vegetarian Restaurant
5,Marol,Western Suburbs,Indian Restaurant,Hotel,Ice Cream Parlor,Asian Restaurant,Farmers Market,Boat or Ferry,Coffee Shop,Diner,Snack Place,Department Store
15,I.C. Colony,Western Suburbs,Indian Restaurant,Fast Food Restaurant,Bar,Coffee Shop,Chinese Restaurant,Dessert Shop,Park,Bakery,Garden Center,BBQ Joint
20,Jogeshwari West,Western Suburbs,Indian Restaurant,Ice Cream Parlor,Asian Restaurant,Dessert Shop,Chinese Restaurant,Mughlai Restaurant,Café,Men's Store,Bakery,Fuel Station
21,Juhu,Western Suburbs,Indian Restaurant,Coffee Shop,Movie Theater,Fast Food Restaurant,Flower Store,Café,Vegan and Vegetarian Restaurant,Bus Station,Shopping Mall,Clothing Store
23,Poisar,Western Suburbs,Indian Restaurant,Rail Station,Arts and Entertainment,Men's Store,Bakery,Restaurant,Electronics Store,Mexican Restaurant,Bus Line,Miscellaneous Store
29,Sunder Nagar,Western Suburbs,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Café,Bakery,Restaurant,Breakfast Spot,Pizzeria,Movie Theater,Seafood Restaurant
30,Kalina,Western Suburbs,Indian Restaurant,Bar,Performing Arts Venue,BBQ Joint,Fuel Station,Pet Supplies Store,Chinese Restaurant,Sandwich Spot,Fast Food Restaurant,Coffee Shop
31,Irla,Western Suburbs,Indian Restaurant,Café,Coffee Shop,Bakery,Scenic Lookout,Chinese Restaurant,Clothing Store,Bus Station,Seafood Restaurant,Flea Market
32,Vile Parle,Western Suburbs,Indian Restaurant,Hotel,Café,Fast Food Restaurant,Sandwich Spot,Coffee Shop,Bar,Bakery,Donut Shop,Tea Room


In [97]:
mum_merged.loc[mum_merged['Cluster Labels'] == 4, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,Navy Nagar,South Mumbai,Convenience Store,Lounge,Arts and Entertainment,Airport,Music Venue,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument,Mosque,Motorcycle Dealership


In [98]:
new_restaurant_neighborhoods = mum_merged.loc[mum_merged['Cluster Labels'] == 1, mum_merged.columns[[0, 1, 2, 3,4] + list(range(5, mum_merged.shape[1]))]]
new_restaurant_neighborhoods.head(30)

,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Western Suburbs,19.1293,72.84644,1,Indian Restaurant,Coffee Shop,Bakery,Pizzeria,Asian Restaurant,Travel and Transportation,Sandwich Spot,Bar,Metro Station,Tea Room
2,D.N. Nagar,Western Suburbs,19.124085,72.83248,1,Bar,Indian Restaurant,Pub,Pizzeria,Chinese Restaurant,Juice Bar,Snack Place,Vegan and Vegetarian Restaurant,Coffee Shop,Bowling Alley
3,Four Bungalows,Western Suburbs,19.1264,72.8242,1,Pub,Café,Indian Restaurant,Chinese Restaurant,Bar,Ice Cream Parlor,Seafood Restaurant,Pizzeria,Coffee Shop,Vegan and Vegetarian Restaurant
4,Lokhandwala,Western Suburbs,19.14321,72.82497,1,Indian Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Coffee Shop,Bakery,Bar,Fast Food Restaurant,Asian Restaurant
6,Sahar,Western Suburbs,19.109461,72.852807,1,Fast Food Restaurant,Indian Restaurant,Hotel,Snack Place,Tea Room,Café,Vegan and Vegetarian Restaurant,Movie Theater,Chinese Restaurant,Shopping Mall
7,Seven Bungalows,Western Suburbs,19.129052,72.8214,1,Café,Bar,Pub,Indian Restaurant,Ice Cream Parlor,Chinese Restaurant,Seafood Restaurant,Pizzeria,Vegan and Vegetarian Restaurant,Coffee Shop
8,Versova,Western Suburbs,19.13769,72.81348,1,Café,Ice Cream Parlor,Beach,Bar,Coffee Shop,Chinese Restaurant,Pub,Smoke Shop,Frozen Yogurt Shop,Music Venue
9,Mira Road,Western Suburbs,19.262969,72.87265,1,Movie Theater,Convenience Store,Bar,Gift Store,Food Truck,Food Court,Multicuisine Indian Restaurant,Fast Food Restaurant,Electronics Store,College and University
10,Bhayandar,Western Suburbs,19.30743,72.85184,1,Bakery,Restaurant,Travel and Transportation,Bike Rental,Electronics Store,Diner,Rail Station,Food Truck,Indian Restaurant,Ice Cream Parlor
12,Bandstand Promenade,Western Suburbs,19.04745,72.82065,1,Coffee Shop,Café,Chinese Restaurant,Indian Restaurant,Snack Place,Tea Room,Pizzeria,Scenic Lookout,Performing Arts Venue,Deli


In [99]:
map_res_locations = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, location, neighborhood in zip(new_restaurant_neighborhoods['Latitude'], new_restaurant_neighborhoods['Longitude'],
                                            new_restaurant_neighborhoods['Location'], new_restaurant_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, location)
    folium.Marker([lat, lng], popup='{} has geographical coordinates ({:.4f}, {:.4f})'.format(label, lat, lng),
                  icon=folium.Icon(color='lightred'), tooltip=label).add_to(map_res_locations)

    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_res_locations)

map_res_locations